In [3]:
import pandas as pd

# Load the datasets
file1 = "~/AQIprediction/daily_climate.csv"  # Replace with the actual file path for the weather dataset
file2 = "~/AQIprediction/beijing-air-quality.csv"  # Replace with the actual file path for the pollution dataset

# Read the datasets into pandas DataFrames
weather_df = pd.read_csv(file1)
pollution_df = pd.read_csv(file2)

In [4]:
# Convert `DAY` and `date` columns to the same format (YYYY-MM-DD)
weather_df['DAY'] = pd.to_datetime(weather_df['DAY'], format='%Y-%m-%d')
pollution_df['date'] = pd.to_datetime(pollution_df['date'], format='%m/%d/%Y')

# Rename `date` in pollution_df to `DAY` for merging
pollution_df.rename(columns={'date': 'DAY'}, inplace=True)

In [5]:
# Split `WND` into `wnd_direc` and `wnd_scale`
def process_wnd(wnd_value):
    if isinstance(wnd_value, str) and wnd_value.startswith("(") and wnd_value.endswith(")"):
        # Parse the tuple as a string
        try:
            wnd_direc, wnd_scale = eval(wnd_value)
            return pd.Series({'wnd_direc': float(wnd_direc), 'wnd_scale': float(wnd_scale)})
        except:
            return pd.Series({'wnd_direc': None, 'wnd_scale': None})
    else:
        return pd.Series({'wnd_direc': None, 'wnd_scale': None})

In [6]:
# Apply the function to separate WND values
weather_df[['wnd_direc', 'wnd_scale']] = weather_df['WND'].apply(process_wnd)

# Drop the original `WND` column
weather_df.drop(columns=['WND'], inplace=True)

In [8]:
# Merge the two datasets on the `DAY` column
combined_df = pd.merge(weather_df, pollution_df, on='DAY', how='inner')

# Rename all columns to lowercase after the merge
combined_df.columns = combined_df.columns.str.lower()

# Save the combined DataFrame to a new CSV file
output_file = "~/AQIprediction/combined_data.csv"
combined_df.to_csv(output_file, index=False)
print(f"Combined data with lowercase column names saved to {output_file}")

Combined data with lowercase column names saved to ~/AQIprediction/combined_data.csv
